In [3]:
# 1. CÀI ĐẶT THƯ VIỆN (chạy 1 lần thôi)
!pip install pycryptodome

# 2. IMPORT
import os, socket, json, base64, hashlib, threading, time
from datetime import datetime
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA512
from Crypto.Random import get_random_bytes

# 3. TẠO KHÓA RSA nếu chưa có
def generate_keys():
    if os.path.exists("sender_private.pem") and os.path.exists("receiver_private.pem"):
        print("Đã có khóa RSA, không tạo lại.")
        return
    sender_key = RSA.generate(2048)
    receiver_key = RSA.generate(2048)

    with open("sender_private.pem", "wb") as f:
        f.write(sender_key.export_key())
    with open("sender_public.pem", "wb") as f:
        f.write(sender_key.publickey().export_key())

    with open("receiver_private.pem", "wb") as f:
        f.write(receiver_key.export_key())
    with open("receiver_public.pem", "wb") as f:
        f.write(receiver_key.publickey().export_key())
    print("Đã tạo khóa RSA sender và receiver.")

generate_keys()

# 4. Receiver - lắng nghe
def receiver():
    def unpad(s):
        return s[:-s[-1]]

    s = socket.socket()
    s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    s.bind(('localhost', 8003))
    s.listen(1)
    print("Receiver: Đang chờ kết nối...")

    conn, _ = s.accept()
    print("Receiver: Đã kết nối sender")

    # Handshake
    hello = conn.recv(1024).decode()
    if hello != "Hello!":
        conn.send(b"NACK")
        conn.close()
        print("Receiver: Không nhận được Hello!")
        return
    conn.send(b"Ready!")
    print("Receiver: Đã nhận Hello!, gửi Ready!")

    # Nhận gói tin JSON
    data = conn.recv(16384)
    packet = json.loads(data.decode())

    # Giải mã session key bằng private RSA OAEP + SHA512
    with open("receiver_private.pem", "rb") as f:
        privkey = RSA.import_key(f.read())
        cipher_rsa = PKCS1_OAEP.new(privkey, hashAlgo=SHA512)
        try:
            session_key = cipher_rsa.decrypt(base64.b64decode(packet['enc_session_key']))
        except Exception as e:
            conn.send(b"NACK")
            print(f"Receiver: Lỗi giải mã khóa phiên: {e}")
            conn.close()
            return

    iv = base64.b64decode(packet['iv'])
    cipher = base64.b64decode(packet['cipher'])
    received_hash = packet['hash']
    signature = base64.b64decode(packet['sig'])
    pwd_hash_received = packet['pwd']

    # Kiểm mật khẩu
    password_correct_hash = hashlib.sha256("BacSi123".encode()).hexdigest()
    if pwd_hash_received != password_correct_hash:
        conn.send(b"NACK")
        print("Receiver: Mật khẩu không hợp lệ")
        conn.close()
        return

    # Kiểm chữ ký metadata
    metadata = packet['metadata']
    metadata_str = json.dumps(metadata, sort_keys=True).encode()
    with open("sender_public.pem", "rb") as f:
        pubkey = RSA.import_key(f.read())
        try:
            pkcs1_15.new(pubkey).verify(SHA512.new(metadata_str), signature)
        except (ValueError, TypeError):
            conn.send(b"NACK")
            print("Receiver: Chữ ký không hợp lệ")
            conn.close()
            return

    # Kiểm hash toàn vẹn
    calc_hash = hashlib.sha512(iv + cipher).hexdigest()
    if calc_hash != received_hash:
        conn.send(b"NACK")
        print("Receiver: Hash không khớp")
        conn.close()
        return

    # Giải mã AES-CBC PKCS7
    cipher_aes = AES.new(session_key, AES.MODE_CBC, iv)
    plaintext = unpad(cipher_aes.decrypt(cipher))

    with open("received_medical_record.txt", "wb") as f:
        f.write(plaintext)

    conn.send(b"ACK")
    print("Receiver: File hợp lệ, đã lưu")

    conn.close()

# 5. Sender - gửi file với nội dung cố định
def sender():
    # Nội dung cố định thay cho nội dung ngẫu nhiên
    fixed_content = """Bệnh án bệnh nhân:
- Họ tên: Nguyễn Văn A
- Ngày sinh: 01/01/1980
- Chẩn đoán: Viêm phổi cấp
- Ghi chú: Cần theo dõi sát tình trạng hô hấp.
"""
    print("Nội dung được sử dụng:\n", fixed_content)
    with open("medical_record.txt", "w", encoding="utf-8") as f:
        f.write(fixed_content)
    print("Sender: medical_record.txt đã được tạo với nội dung cố định.")

    time.sleep(0.5) # Đợi receiver sẵn sàng
    s = socket.socket()
    s.connect(('localhost', 8003))

    # Handshake
    s.send(b"Hello!")
    ready = s.recv(1024).decode()
    if ready != "Ready!":
        print("Sender: Không nhận được Ready!")
        s.close()
        return
    print("Sender: Đã nhận Ready!")

    # Tạo session key AES-128
    session_key = get_random_bytes(16)

    # Tạo metadata
    metadata = {
        "filename": "medical_record.txt",
        "timestamp": datetime.now().isoformat(),
        "record_id": "MR" + datetime.now().strftime("%Y%m%d%H%M%S")
    }
    metadata_str = json.dumps(metadata, sort_keys=True).encode()

    # Ký metadata bằng RSA sender private SHA512
    with open("sender_private.pem", "rb") as f:
        privkey = RSA.import_key(f.read())
        signature = pkcs1_15.new(privkey).sign(SHA512.new(metadata_str))

    # Mã hóa session key RSA receiver public OAEP SHA512
    with open("receiver_public.pem", "rb") as f:
        receiver_pub = RSA.import_key(f.read())
        cipher_rsa = PKCS1_OAEP.new(receiver_pub, hashAlgo=SHA512)
        enc_session_key = cipher_rsa.encrypt(session_key)

    # Đọc file plaintext
    with open("medical_record.txt", "rb") as f:
        plaintext = f.read()

    # AES-CBC mã hóa với IV random, padding PKCS7
    iv = get_random_bytes(16)
    cipher_aes = AES.new(session_key, AES.MODE_CBC, iv)
    pad_len = 16 - len(plaintext) % 16
    padded_plaintext = plaintext + bytes([pad_len]) * pad_len
    ciphertext = cipher_aes.encrypt(padded_plaintext)

    # Hash iv + ciphertext
    hash_val = hashlib.sha512(iv + ciphertext).hexdigest()

    # Mật khẩu (hardcode "BacSi123")
    password = "BacSi123"
    pwd_hash = hashlib.sha256(password.encode()).hexdigest()

    packet = {
        "iv": base64.b64encode(iv).decode(),
        "cipher": base64.b64encode(ciphertext).decode(),
        "hash": hash_val,
        "sig": base64.b64encode(signature).decode(),
        "pwd": pwd_hash,
        "metadata": metadata,
        "enc_session_key": base64.b64encode(enc_session_key).decode()
    }

    s.send(json.dumps(packet).encode())
    response = s.recv(1024)
    print("Sender nhận phản hồi:", response.decode())
    s.close()

# 6. CHẠY: dùng threading chạy receiver trước rồi sender
receiver_thread = threading.Thread(target=receiver, daemon=True)
receiver_thread.start()

time.sleep(1) # Đợi receiver chạy xong

sender()

# Đợi receiver xử lý xong mới kết thúc chương trình (nếu cần)
time.sleep(1)
print("Kết thúc phiên gửi nhận.")

Đã có khóa RSA, không tạo lại.
Receiver: Đang chờ kết nối...
Nội dung được sử dụng:
 Bệnh án bệnh nhân:
- Họ tên: Nguyễn Văn A
- Ngày sinh: 01/01/1980
- Chẩn đoán: Viêm phổi cấp
- Ghi chú: Cần theo dõi sát tình trạng hô hấp.

Sender: medical_record.txt đã được tạo với nội dung cố định.
Receiver: Đã kết nối sender
Receiver: Đã nhận Hello!, gửi Ready!
Sender: Đã nhận Ready!
Receiver: File hợp lệ, đã lưu
Sender nhận phản hồi: ACK
Kết thúc phiên gửi nhận.
